In [1]:
#Author: @michaelbrink
#Org: BalloonBox Inc.

In [2]:
from bs4 import BeautifulSoup
import requests
import json
from html.parser import HTMLParser
import urllib.request
import string
import random
import re
import pandas as pd

In [3]:
#Need a JS enabled web service
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import selenium.webdriver.chrome.service as service
from selenium.webdriver.chrome.options import Options
import time

In [4]:
options = Options()

In [5]:
options.add_argument("--disable-notifications")

In [6]:
driver = webdriver.Chrome('/Users/hongbinlin/Downloads/chromedriver',options=options)

# Helper Functions

In [7]:
def ceoFounder():
    driver.find_element_by_xpath(
        "//div[@class='input__inputWrapper--3SQdx searchFilters__filter--3igRc input__input--34yMC input__checkbox--1434-']/label[@for='ceofounder']").click()
    time.sleep(15)

In [8]:
def femaleCEO():
    driver.find_element_by_xpath(
        "//div[@class='input__inputWrapper--3SQdx searchFilters__filter--3igRc input__input--34yMC input__checkbox--1434-']/label[@for='ceowoman']").click()
    time.sleep(15)

In [9]:
def pageTurner():
    driver.find_element_by_xpath("//div[@class='-next']").click()
    time.sleep(15)

## Scrape rows of companies data

In [10]:
def contentScraper():
    content = BeautifulSoup(driver.page_source, 'html.parser')
    example = content.find_all('a', attrs={'class':'searchResults__cellWrapper--39MAj'})
    example = [item.string for item in example] #return a list of all singular elements
    example = [example[i:i + 11] for i in range(0, len(example), 11)]
    return(example)

## Creating a nested frame of data

## Loop to scrape

In [11]:
 def web_scraping(func=0):
    #date range = 2017 - 2020
    #row range = 10 - 100
    #page range = depends on how many pages it has
    # func = ceoFounder or femaleCEO or empty

    outputContent = []

    ## Pilot model
    for year in range(2017, 2021): 
        
        URL = "https://fortune.com/fortune500/"+str(year)+"/search/"
        driver.get(URL)
        time.sleep(15)
        print ("Year " + str(year) + " active!")
        
        # Check if it has the filter
        try:
            driver.find_element_by_xpath("//label[@for='ceowoman']")
            check_bool = True
        except:
            check_bool = False
        print(check_bool)
        print(func!=0)
        if (not check_bool) and (func!=0):
            print('Skip this year')
            continue
        else:
            print('Continue this year')
            pass

        # Set any desired filters
        if func!=0:
            func()
            print ("Filter Actived!")
        
        # expand rows to maximum
        driver.find_element_by_xpath("//select[@aria-label='rows per page']/option[@value='100']").click()
        print ("Rows extended")
        time.sleep(15)

        # Locate the number of page
        pages  = driver.find_element_by_xpath("//span[@class='-pageInfo']/span[@class='-totalPages']").text
        pages = int(pages)
        print("The total number of pages is: ",pages)
        
        for page in range (1, pages+1):

            outputContent.append((year, page, contentScraper()))
            print("Page " + str(page) + " completed")

            #increment/update the page
            pageTurner()
            
        print ("Year " + str(year) + " completed")

    return(outputContent)

In [12]:
content = web_scraping(ceoFounder)

Year 2017 active!
False
True
Skip this year
Year 2018 active!
True
True
Continue this year
Filter Actived!
Rows extended
The total number of pages is:  1
Page 1 completed
Year 2018 completed
Year 2019 active!
True
True
Continue this year
Filter Actived!
Rows extended
The total number of pages is:  1
Page 1 completed
Year 2019 completed
Year 2020 active!
True
True
Continue this year
Filter Actived!
Rows extended
The total number of pages is:  1
Page 1 completed
Year 2020 completed


## Convert nested list to dataframe

In [13]:
tempcontent = content.copy()

In [14]:
columns =['Rank',
             'Name',
             'Revenue ($M)',
             'Revenue % change', 
             'Profit ($M)',
             'Profit % change',
             'Assets ($M)',
             'Market Value ($M)',
             'Change in rank (1000)',
             'Employees',
             'Change in rank (500)',
             'Year']

In [15]:
df = pd.DataFrame(columns=columns)

In [16]:
df

,Rank,Name,Revenue ($M),Revenue % change,Profit ($M),Profit % change,Assets ($M),Market Value ($M),Change in rank (1000),Employees,Change in rank (500),Year


In [17]:
''' !!!!!!! For ceoFounder and female CEO !!!!!!! '''
# 3 levels
# tempContent[i][0] == year ; i = 0, 1, 2 
# tempContent [i][2][j] == row ; j = 0, 1, 2, ..., n
# tempcontent [i][2][j][k] == element ; k = range (0, 11) ## Will not use this, grab entire

counter = 0
for i in range(0, 3):
    for j in range (0, len(tempcontent[i][2])):
        tempcontent[i][2][j].append(tempcontent[i][0])
        df.loc[counter] = tempcontent[i][2][j]
        counter +=1

In [18]:
''' !!!!!!! For no filter !!!!!!! '''
# 3 levels
# tempContent[i][0] == year ; i = 0, 1, 2 
# tempContent [i][2][j] == row ; j =  0, 1, 2, ..., n
# tempcontent [i][2][j][k] == element ; k = range (0, 11) ## Will not use this, grab entire

# counter = 0
# for i in range(0, 40):
#     for j in range (0, len(tempcontent[i][2])):
#         tempcontent[i][2][j].append(tempcontent[i][0])
#         df.loc[counter] = tempcontent[i][2][j]
#         counter +=1

' !!!!!!! For no filter !!!!!!! '

In [19]:
content[0][2][0]

['22',
 'Alphabet',
 '$110,855.00',
 '22.8%',
 '$12,662.00',
 '-35%',
 '$197,295.00',
 '$719,123.60',
 '5',
 '80,110',
 '5',
 2018]

In [20]:
tempcontent[0][2][0]

['22',
 'Alphabet',
 '$110,855.00',
 '22.8%',
 '$12,662.00',
 '-35%',
 '$197,295.00',
 '$719,123.60',
 '5',
 '80,110',
 '5',
 2018]

In [21]:
df.shape

(107, 12)

In [22]:
df.head()

,Rank,Name,Revenue ($M),Revenue % change,Profit ($M),Profit % change,Assets ($M),Market Value ($M),Change in rank (1000),Employees,Change in rank (500),Year
0,22,Alphabet,"$110,855.00",22.8%,"$12,662.00",-35%,"$197,295.00","$719,123.60",5,"80,110",5,2018
1,50,FedEx,"$60,319.00",19.8%,"$2,997.00",64.7%,"$48,552.00","$64,161.00",8,"357,000",8,2018
2,76,Facebook,"$40,653.00",47.1%,"$15,934.00",56%,"$84,524.00","$464,190.10",22,"25,105",22,2018
3,101,Capital One Financial,"$29,999.00",9%,"$1,982.00",-47.2%,"$365,693.00","$46,583.90",-1,"49,300",-1,2018
4,231,L Brands,"$12,632.40",0.5%,$983,-15.1%,"$8,149.00","$10,655.20",-11,"59,200",-11,2018


# Save dataset into csv files

In [23]:
# df.to_excel("forbes2017-2020_femaleceo.xlsx")
# df.to_excel("forbes2017-2020_ceofounder.xlsx")
# df.to_excel("forbes2017-2020_nofilter.xlsx")